<a href="https://colab.research.google.com/github/Shopping-Yuan/bert_project/blob/main/keyword_sorting/bert_keyword_sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymysql

     |████████████████████████████████| 51kB 6.0MB/s 


In [ ]:
import LinkMariaDB
import jieba.analyse
import os
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
from jieba.analyse import extract_tags

In [ ]:
# 連 Server 依日期分別抓取訓練集和測試集
hostName = '34.80.102.35'
portNum = 3306
DBName = 'db_blogler'
username = 'shop'
password = '12345678'
CharSet = "utf8"

DB = LinkMariaDB.OpMariaSQL(hostName, portNum, username, password, DBName, CharSet)
raw_data_train = DB.IntputFindCmd(f"SELECT content FROM main WHERE (label_1 = '1' OR label_2 = '1') AND pub_month <= 202001")
raw_data_test = DB.IntputFindCmd(f"SELECT content FROM main WHERE (label_1 = '1' OR label_2 = '1') AND pub_month > 202001")

IntputFindCmd:
 SELECT content FROM main WHERE (label_1 = '1' OR label_2 = '1') AND pub_month <= 202001
IntputFindCmd:
 SELECT content FROM main WHERE (label_1 = '1' OR label_2 = '1') AND pub_month > 202001


In [ ]:
#準備關鍵詞列表,將關鍵詞表中的字轉換為簡體
!pip install opencc-python-reimplemented
from opencc import OpenCC
import re
import jieba.analyse
import os
from urllib.request import urlretrieve
if not os.path.exists ("dict.txt.big"):
  url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
  urlretrieve(url, "dict.txt.big")
jieba.set_dictionary ('dict.txt.big')
 
cc = OpenCC('t2s')
 
keyword_list_convert = []

with open('keyword_list.txt', 'r') as f:
  t = f.readlines()
  for i in t:
    keyword_list_convert.append(cc.convert(i))
with open("keyword_list_convert.txt", "w") as output:
  for item in keyword_list_convert:
    output.write("%s" % item)
medium_dict_path = "keyword_list_convert.txt"
if os.path.exists(medium_dict_path):
    jieba.load_userdict(medium_dict_path)
keyword_list = []
for n in keyword_list_convert:
  keyword_list.append(n.strip("\n"))
jieba.re_han_default = re.compile('(.+)', re.U)

Building prefix dict from /content/dict.txt.big ...
Loading model from cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.027 seconds.
Prefix dict has been built successfully.


In [ ]:
#過濾文章用標點
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')

In [ ]:
import numpy as np
def data(x):
  # cut with jieba 且移除標點符號
  words = ",".join(filter(lambda x: x not in punct, jieba.cut(x)))
  return words

In [ ]:
#將訓練集與測試集的資料轉為簡體
df_train = pd.DataFrame(raw_data_train,columns = ["content"])
df_test = pd.DataFrame(raw_data_test,columns = ["content"])
df_convert_train = pd.DataFrame(df_train["content"].apply(cc.convert),columns = ["content"])
df_convert_test = pd.DataFrame(df_test["content"].apply(cc.convert),columns = ["content"])
train = np.array(df_convert_train["content"].apply(data))
test = np.array(df_convert_test["content"].apply(data))
total_df_train = pd.DataFrame(train,columns = ["content"])
total_df_test = pd.DataFrame(test,columns = ["content"])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import math
from sklearn.preprocessing import normalize

In [ ]:
stop_words_list = []
with open('stop_words.txt', 'r') as f:
  stop_words_list = [line.strip() for line in f]
  t = f.readlines()
  for i in t:
    stop_words_list.append(cc.convert(i))

In [ ]:
from collections import Counter
def merge_dict(x,y):
  X,Y=Counter(x),Counter(y)
  z=dict(X+Y)
  return(z)

In [ ]:
# 安裝keras-bert函式庫
!pip -q install keras-bert
!pip -q install keras-transformer
!unzip drive/MyDrive/multi_cased_L-12_H-768_A-12.zip

Archive:  drive/MyDrive/multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  


In [ ]:
# SEQ_LEN: 取原文的多少個字, 建議至少256以上, 512是最大值, 但是會導致模型太大而RAM不夠用(超過的"後文"會被刪去)
SEQ_LEN = 512

In [ ]:
# 讀取預訓練模型
import os
pretrained_path = 'multi_cased_L-12_H-768_A-12'
# pretrained_path = "publish"
# 參數設定
config_path = os.path.join(pretrained_path, 'bert_config.json')
# 模型本身
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
# 所有支持的字彙
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [ ]:
# BERT 透過 Subword 的方式讓辭典長度最大就是 21128
token_dict = {}
with open(vocab_path, 'r', encoding='utf8') as f:
    for line in f.readlines():
        token = line.strip()
        token_dict[token] = len(token_dict)
print("辭典長度:", len(token_dict))

辭典長度: 119547


In [ ]:
#注意單字表是從1開始和Token的值會有差異
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)

In [ ]:
#對文章進行分詞 並得到bert的二項輸入
import numpy as np
def tokenize_data(x):
  # cut with jieba 且移除標點符號
  words = "".join(jieba.cut(x))
  ids, segments = tokenizer.encode(words, max_len=SEQ_LEN)
  ids = np.array(ids)
  segments = np.array(segments, dtype ='f2')
  return ids, segments

In [ ]:
#準備資料流
import pandas as pd
def prepare_input(x):
  x_tokenize = np.array(x.apply(tokenize_data)) 
  en_r = []
  seq_r = []
  for n in range(x_tokenize.shape[0]):
    en_r.append(x_tokenize[n][0])
    seq_r.append(x_tokenize[n][1])
  train_list_x = [np.array(en_r), np.array(seq_r)]
  return train_list_x

In [ ]:
#準備標籤step1,先標記關鍵詞的位置
def word_position(content,keyword=keyword_list):
  keyword_set = set(keyword)
  content_word_segment= []
  content = jieba.lcut(content)
  words = []
  for word in content:
        if word not in punct:          
            words.append(word)
  content = words
  for n in content:
    test, se = tokenizer.encode(n)
    len_of_token = (len(test)-2)
    if len_of_token>0:
      if n in keyword_set:
        content_word_segment.append(100+len_of_token)
      else:
        try:
          x = n.lower()
          if x in keyword_set:
            content_word_segment.append(100+len_of_token)
          else:
            content_word_segment.append(len_of_token)
        except:
          content_word_segment.append(len_of_token)
  return np.array(content_word_segment)

In [ ]:
#準備標籤step2,輸出元素為0和1的list,1代表該位置的字為關鍵詞的一部分
def prepare_label(content_word_segment:np.array, sq_le = SEQ_LEN):
  content_char_segment= np.array([1])
  for n in content_word_segment:
    if n >100:
      content_char_segment = np.concatenate([content_char_segment,np.ones(n-100)])
    else:
      content_char_segment = np.concatenate([content_char_segment,np.zeros(n)])
  # content_char_segment = np.concatenate([content_char_segment,np.zeros(1)])
  content_char_segment = np.concatenate([content_char_segment,np.zeros(SEQ_LEN)])
  content_char_segment = content_char_segment[:SEQ_LEN]
  return content_char_segment

In [ ]:
#比較BERT預測的重要度序列和對應的預測文章,生成結果
def retrieve_keyword(content,content_predict_weight):
  predict_keyword_value_dic = {}
  len_of_token_list = []
  index = 1

  content = jieba.lcut(content)
  words = []
  for word in content:
        if word not in punct:          
            words.append(word)
  content = words
  for n in content:
    test, se = tokenizer.encode(n)
    len_of_token_list.append(len(test)-2)
    # print(n,len(test)-2)
    predict_keyword_value = 0
    for m in range(len(test)-2):
        if index<SEQ_LEN:
          predict_keyword_value += content_predict_weight[index]
          index += 1
          predict_keyword_value_dic[n] = predict_keyword_value/(len(test)-2)
  return predict_keyword_value_dic

In [ ]:
SEQ_LEN = 512

In [ ]:
#x輸入是DataFrame:total_df
#y輸入是list:keyword_list_convert

In [ ]:
df_y_train = total_df_train["content"].apply(word_position)
df_y_test = total_df_test["content"].apply(word_position)

In [ ]:
train_x = total_df_train["content"]
train_y = df_y_train.apply(prepare_label)
test_x = total_df_test["content"]
test_y = df_y_test.apply(prepare_label)

In [ ]:
train_list_x = prepare_input(train_x)
test_list_x = prepare_input(test_x)

In [ ]:
#讀取並微調模型
import tensorflow.keras
from keras_bert import load_trained_model_from_checkpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional,LSTM

#1.讀取模型
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=False,
    trainable=False,
    seq_len=SEQ_LEN,
    )

#2.微調模型
inputs = model.inputs[:2]
x = model.output
bi_lstm = Bidirectional(LSTM(64, return_sequences=True))(x)
outputs = Dense(units=2, activation='softmax')(bi_lstm)
model = tensorflow.keras.models.Model(inputs, outputs)

#3.設置訓練方式
Optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Optimizer,
    metrics=['sparse_categorical_accuracy']
)
# model.summary()

In [ ]:
from keras.utils import np_utils
train_y = np.array(train_y.tolist())
train_y = pd.DataFrame(train_y)
train_y = train_y.astype('float32')

In [ ]:
# BATCH_SIZE: 每次多少個BATCH再做調整, 如果RAM不夠用就調小
EPOCHS = 5
BATCH_SIZE = 16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model.fit(
    train_list_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    callbacks=[
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=3, restore_best_weights=True)],
)

Epoch 1/5
258/258 [==============================] - 152s 497ms/step - loss: 0.4320 - sparse_categorical_accuracy: 0.8167 - val_loss: 0.3549 - val_sparse_categorical_accuracy: 0.8575
Epoch 2/5
258/258 [==============================] - 123s 475ms/step - loss: 0.3626 - sparse_categorical_accuracy: 0.8508 - val_loss: 0.3497 - val_sparse_categorical_accuracy: 0.8578
Epoch 3/5
258/258 [==============================] - 123s 477ms/step - loss: 0.3620 - sparse_categorical_accuracy: 0.8500 - val_loss: 0.3463 - val_sparse_categorical_accuracy: 0.8591
Epoch 4/5
258/258 [==============================] - 123s 475ms/step - loss: 0.3645 - sparse_categorical_accuracy: 0.8487 - val_loss: 0.3439 - val_sparse_categorical_accuracy: 0.8596
Epoch 5/5
258/258 [==============================] - 123s 475ms/step - loss: 0.3586 - sparse_categorical_accuracy: 0.8523 - val_loss: 0.3420 - val_sparse_categorical_accuracy: 0.8598


In [ ]:
test_y = np.array(test_y.tolist())

test_y = pd.DataFrame(test_y)

test_y = test_y.astype('float32')

In [ ]:
px = model.predict(test_list_x)

In [ ]:
predict_list = []
for n in px:
  predict_list_n = []
  for m in n:
   predict_list_n.append(m[1]/(m[0]+m[1]))
  predict_list.append(predict_list_n)
len(predict_list)

1145

In [ ]:
stopword_set = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？&｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')
with open('stop_words.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = cc.convert(data)
        data = data.strip("\n")
        stopword_set.add(data)
# stopword_set

In [ ]:
def sort_key(x):
  return x[1]
keyword_dic = {}
for n in range(len(predict_list)):
  pn_dic =retrieve_keyword(test_x.iloc[n],predict_list[n])
  pn_list = sorted(pn_dic.items(), key=lambda x:x[1], reverse=True)
  pn_list_new = []
  for m in pn_list:
    append = True
    if len(m[0]) >= 4 :

      if m[0] in stopword_set and m[0] in keyword_list_convert:
        append = False
      elif m[0] not in keyword_list_convert:
        for n in m[0]:
          if n in stopword_set:
            append = False
            if append == True:
              new_content = jieba.lcut(m[0])
              for i in new_content:
                if i in stopword_set:
                  append = False
                  break
      else: append = False
    else:
      append = False
    if append == True:
      pn_list_new.append(m)
  # pn_list_new = pn_list_new.sort(key = sort_key,reverse = True)
  keyword_dic = merge_dict(keyword_dic,dict(pn_list_new))
keyword_dic

{'什么时候': 1.1076325019821525,
 'lidemy': 0.12395427624384563,
 'leetcode': 1.4825142174959183,
 'https': 28.553029842674732,
 '换句话说': 0.6564901396632195,
 'list': 2.799435034394264,
 '应用程式': 5.82672374881804,
 'todo': 0.06661038845777512,
 'JavaScript': 11.334809105843307,
 '详细资料': 0.36483503319323063,
 '数据模型': 0.4253608491271734,
 'AARRR': 0.1321996053059896,
 '生命周期': 0.7523399963974953,
 '电子商务': 0.48929811641573906,
 '业务知识': 0.11445854604244232,
 '数据分析': 3.5106791388243437,
 '一段时间': 2.5420600110664964,
 '问题进行': 0.17787139862775803,
 '市场需求': 0.15170761942863464,
 'TIOBE': 0.1516822651028633,
 '头把交椅': 0.12313860841095448,
 '事实证明': 0.11707754619419575,
 '有史以来': 0.11123639158904552,
 'Developer': 0.8249490484595299,
 'HackerRank': 0.10653067380189896,
 'Skills': 0.10356064140796661,
 'Python': 16.476777881383892,
 '身兼数职': 0.10095412656664848,
 '领域专家': 0.25505949929356575,
 '开发人员': 1.788841588422656,
 'Java': 4.933448452502489,
 '前端开发': 0.24318679422140121,
 'report': 0.3223402798175812,
 

In [ ]:
sorted(keyword_dic.items(), key=lambda x:x[1],reverse=True)

[('https', 28.553029842674732),
 ('function', 18.250940293073654),
 ('Python', 16.476777881383892),
 ('JavaScript', 11.334809105843307),
 ('Swift', 9.769431645671531),
 ('data', 9.191191650927067),
 ('Xcode', 8.861409030854702),
 ('Google', 8.354529611766338),
 ('with', 7.983256109058857),
 ('作业系统', 7.853601444512606),
 ('SwiftUI', 7.741979923099279),
 ('view', 7.4817656725645065),
 ('python', 6.9430550436178855),
 ('code', 6.915855623781681),
 ('github', 6.81311268856128),
 ('index', 5.986184410750866),
 ('HTML', 5.93333874642849),
 ('this', 5.907745487987995),
 ('from', 5.863743171095848),
 ('应用程式', 5.82672374881804),
 ('Linux', 5.749897835155328),
 ('Windows', 5.518788680434227),
 ('value', 5.497526794672012),
 ('image', 5.159724846482277),
 ('server', 5.137190438807011),
 ('参考资料', 5.096052619628608),
 ('google', 5.0524574145674706),
 ('class', 5.017775423824787),
 ('html', 5.00600865483284),
 ('Java', 4.933448452502489),
 ('array', 4.9246387630701065),
 ('type', 4.810215093195438),